<a href="https://colab.research.google.com/github/amalrik1/develop_ml_course/blob/main/practice2/adult_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пример демонстрирует импорт, предобработку и построение модели машинного обучения для датасета [Adult](https://www.kaggle.com/datasets/wenruliu/adult-income-dataset).

Итоговая модель сохраняется как сериализованный pickle-объект.

Датасет, модель и эксперимент загружаются в ClearML.
Авторизационные ключи в облако ClearML должны быть ваши личные.

ClearML — это фреймворк для трекинга ML-экспериментов. Это основное его предназначение. Но сейчас функционал ClearML гораздо шире и позволяет:

* Отслеживать метрики, гиперпараметры и артефакты машинного обучения.

* Хранить и предоставлять по запросу модели.

* Хранить датасеты.

* Визуально сравнивать эксперименты.

* Воспроизведить эксперименты.

* Автоматически логировать все действия.

* Настраивать пайплайны обработки данных.

* Визуализировать результаты.

Главный конкурент ClearML – Weights & Biases. Но у ClearML есть два серьезных преимущества:

* Облачной версией могут пользоваться бесплатно даже небольшие команды.

* Есть полноценная локально разворачиваемая версия.

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=
%env CLEARML_API_SECRET_KEY=

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=8L1MUGF3LGJDDME4TIFS4RQXZPENJE
env: CLEARML_API_SECRET_KEY=8Ywh2v3Bd_NLCxJcKNkA_-jIqL-NvjjbXnl0kXgtTZIat5VBBhiZZMsm177b6yqqLEE


In [2]:
from clearml import Task, Logger
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Чтобы начать работать с ClearML инициируем новую сессию:
task = Task.init(
    project_name='Adult model', 
    task_name='Эксперимент  1', 
    tags=['DecisionTreeClassifier'])

ClearML Task: created new task id=099949e435f84b8197b1e9003ffed6a5
2025-11-28 15:31:56,685 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/fc7b3b8e4d4745498b873e9637bb0552/experiments/099949e435f84b8197b1e9003ffed6a5/output/log


ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
2025-11-28 15:32:42,323 - clearml.storage - INFO - Uploading: 5.50MB from d:/code/polytex/4/adults.csv


██████████████████████████████████ 100% | 5.50/5.5 MB [00:03<00:00,  1.81MB/s]: 


ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
ClearML Monitor: Reporting detected, reverting back to iteration based reporting


Загрузим датасет
Помимо загрузки данных в pandas, мы попутно отправляем наши данные в ClearML как Артефакт, просто указав путь к CSV-файлу и задав имя артифакта. Все загруженные артефакты можно найти на вкладке Artifacts, соответствующего эксперимента.

In [4]:

fpath = 'adults.csv'
df_raw = pd.read_csv(fpath)
task.upload_artifact(name='data.raw', artifact_object=fpath)

True

In [5]:
df_raw.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


ClearML может хранить в виде Артифактов практически любые Python-объекты. Здесь мы поместили в Артефакт вывод пандосовского метода describe. 

In [6]:
task.upload_artifact(
    name='eda.describe.object', 
    artifact_object=df_raw.describe(include=object))
task.upload_artifact(
    name='eda.describe.number', 
    artifact_object=df_raw.describe(include=np.number))

True

In [7]:
df = df_raw.drop(['fnlwgt', 'education_num'], axis=1)

In [11]:
col_names = df.columns

for c in col_names:
    df = df.replace("?", np.nan)
df = df.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [12]:
df.replace(['Divorced', 'Married-AF-spouse',
            'Married-civ-spouse', 'Married-spouse-absent',
            'Never-married', 'Separated', 'Widowed'],
           ['divorced', 'married', 'married', 'married',
            'not married', 'not married', 'not married'], inplace=True)

category_col = ['workclass', 'race', 'education', 'marital_status', 'occupation',
                'relationship', 'sex', 'native_country', 'class']
labelEncoder = preprocessing.LabelEncoder()

mapping_dict = {}
for col in category_col:
    df[col] = labelEncoder.fit_transform(df[col])

    le_name_mapping = dict(zip(labelEncoder.classes_,
                               labelEncoder.transform(labelEncoder.classes_)))

    mapping_dict[col] = le_name_mapping
print(mapping_dict)

{'workclass': {' ?': np.int64(0), ' Federal-gov': np.int64(1), ' Local-gov': np.int64(2), ' Never-worked': np.int64(3), ' Private': np.int64(4), ' Self-emp-inc': np.int64(5), ' Self-emp-not-inc': np.int64(6), ' State-gov': np.int64(7), ' Without-pay': np.int64(8)}, 'race': {' Amer-Indian-Eskimo': np.int64(0), ' Asian-Pac-Islander': np.int64(1), ' Black': np.int64(2), ' Other': np.int64(3), ' White': np.int64(4)}, 'education': {' 10th': np.int64(0), ' 11th': np.int64(1), ' 12th': np.int64(2), ' 1st-4th': np.int64(3), ' 5th-6th': np.int64(4), ' 7th-8th': np.int64(5), ' 9th': np.int64(6), ' Assoc-acdm': np.int64(7), ' Assoc-voc': np.int64(8), ' Bachelors': np.int64(9), ' Doctorate': np.int64(10), ' HS-grad': np.int64(11), ' Masters': np.int64(12), ' Preschool': np.int64(13), ' Prof-school': np.int64(14), ' Some-college': np.int64(15)}, 'marital_status': {' Divorced': np.int64(0), ' Married-AF-spouse': np.int64(1), ' Married-civ-spouse': np.int64(2), ' Married-spouse-absent': np.int64(3), 

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X = df.values[:, 0:12]
Y = df.values[:, 12]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
           X, Y, test_size = 0.3, random_state = 100)

In [15]:
task.upload_artifact(name='data.preproc', artifact_object=df)
task.upload_artifact(name='data.train', artifact_object=X_train)
task.upload_artifact(name='label.train', artifact_object=y_train)
task.upload_artifact(name='data.test', artifact_object=X_test)
task.upload_artifact(name='label.test', artifact_object=y_test)

True

Мы сформировали еще три датасета и все три отправили в ClearML как Артефакты. Но в отличии от первого раза, сейчас мы напрямую указали пандосовский датасет. ClearML понимает этот формат и мы можем (частично) посмотреть на датасет прямо из интерфейса ClaerML.

Обучим модель

In [16]:

dt_clf_gini = DecisionTreeClassifier(criterion = "gini",
                                     random_state = 100,
                                     max_depth = 5,
                                     min_samples_leaf = 5)

dt_clf_gini.fit(X_train, y_train)
y_pred_gini = dt_clf_gini.predict(X_test)
acc = accuracy_score(y_test, y_pred_gini)*100 

print ("Decision Tree using Gini Index\nAccuracy is ",
             acc )

Decision Tree using Gini Index
Accuracy is  82.5769467003344


In [17]:
# Инциируем объект логирования
log = Logger.current_logger()
log.report_single_value(name='Accuracy', value=acc)

In [18]:
import pickle

In [19]:
with open("model.pkl", "wb") as f:
    model = pickle.dump(dt_clf_gini, f)

In [20]:
task.upload_artifact(name='model', artifact_object=model)

True

In [21]:
task.close()

Дополнительные материалы

Статья: https://habr.com/ru/articles/691314/

Документация: https://clear.ml/docs/latest/docs/